<a href="https://colab.research.google.com/github/ditarnnt/Telecom-Predic-Churn/blob/main/PP_IBM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##IMPORT LIBRARY

In [ ]:
from google.colab import drive
import pandas as pd #pandas
import numpy as np #numpy
from sklearn.preprocessing import LabelEncoder, OneHotEncoder #encoding data kategorikal
from sklearn.model_selection import train_test_split #train test split

##IMPORT DATASET

In [ ]:
drive.mount('/content/gdrive')

df = pd.read_csv('/content/gdrive/My Drive/TubesAI_folder/IBM_dataset.csv')
df

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,6840-RESVB,Male,0,Yes,Yes,24,Yes,Yes,DSL,Yes,No,Yes,Yes,Yes,Yes,One year,Yes,Mailed check,84.80,1990.5,No
7039,2234-XADUH,Female,0,Yes,Yes,72,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,One year,Yes,Credit card (automatic),103.20,7362.9,No
7040,4801-JZAZL,Female,0,Yes,Yes,11,No,No phone service,DSL,Yes,No,No,No,No,No,Month-to-month,Yes,Electronic check,29.60,346.45,No
7041,8361-LTMKD,Male,1,Yes,No,4,Yes,Yes,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Mailed check,74.40,306.6,Yes


##PREPROCESSING

In [ ]:
#Ini perlu dijalanin sekali aja

index_drop = df.loc[df.TotalCharges == ' '].index.tolist() #Mencari data tidak jelas
df.drop(index=index_drop,inplace=True) #Menghapus data tidak jelas
df.drop(columns=['customerID'],inplace=True) #Menghapus customerID, fitur tidak relevan
df.reset_index(drop=True,inplace=True) #Reset index setelah didrop
df['TotalCharges'] = df['TotalCharges'].astype(float) #Konversi jadi float

In [ ]:
#Membedakan fitur numerik dan kategorikal

def diff(first, second): #fungsi pengurangan list
  '''
  Fungsi yang menghitung difference dari list. Elemen dari list pertama yang ada di list kedua dihilangkan
  tanpa mengubah urutan list pertama.
  Input:
  first - list pertama
  second - list kedua
  Output:
  List yang mengandung difference dari list pertama dan kedua sesuai definisi fungsi.
  '''
  second = set(second)
  return [item for item in first if item not in second]

cols = df.columns.tolist() #kolom-kolom

label = ['Churn']

features = diff(cols,label) #fitur-fitur

num_features = df.select_dtypes(include=np.number).columns.tolist() 
num_features = diff(num_features,label) #fitur-fitur numerik
num_features = diff(num_features,['SeniorCitizen']) #pengecualian

cat_features = diff(features,num_features) #fitur-fitur kategorikal

cols = num_features + cat_features + label #kolom-kolom yang sudah dirapihkan

df = df[cols] #pengurutan kolom dataframe agar rapih

print('Kolom-kolom total:')
print(cols,'\n')
print('Fitur-fitur:')
print(features,'\n')
print('Fitur-fitur numerik:')
print(num_features,'\n')
print('Fitur-fitur kategorikal:')
print(cat_features,'\n')
print('Label:')
print(label,'\n')
print('Dataframe:')
print(df,'\n')

Kolom-kolom total:
['tenure', 'MonthlyCharges', 'TotalCharges', 'gender', 'SeniorCitizen', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod', 'Churn'] 

Fitur-fitur:
['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure', 'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod', 'MonthlyCharges', 'TotalCharges'] 

Fitur-fitur numerik:
['tenure', 'MonthlyCharges', 'TotalCharges'] 

Fitur-fitur kategorikal:
['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod'] 

L

In [ ]:
def encode_cat(df,cat_features):
  '''
  Fungsi untuk meng-encode kolom-kolom kategorikal secara One-Hot Encoding.
  Input: 
  df - DataFrame pandas yang akan di-encode
  cat_features - list berupa fitur-fitur kategorikal yang akan di-encode
  Output:
  DataFrame df yang sudah di-encode fitur-fitur kategorikalnya.
  '''
  df_placeholder = pd.DataFrame(index=df.index.tolist())
  for cat_col in cat_features:
    dum_df = pd.get_dummies(df[cat_col], columns=[cat_col],prefix=cat_col)
    df_placeholder = df_placeholder.join(dum_df)
  df = df.drop(columns=cat_features)
  df = df.join(df_placeholder)
  temp = df['Churn']
  df.drop(columns=['Churn'],inplace=True)
  df = df.join(temp)
  return df

def encode_label(df,label):
  '''
  Fungsi yang meng-encode label dari dataframe.
  Input:
  df - DataFrame yang labelnya akan di-encode
  label - string label
  Output:
  DataFrame df yang labelnya sudah di-encode.
  '''
  labelencoder = LabelEncoder()
  df[label] = labelencoder.fit_transform(df[label])
  return df

#Encode fitur-fitur kategorikal dengan onehot encoder
df = encode_cat(df,cat_features)
#Encode label dengan label encoder
df = encode_label(df,label[0])

df

,tenure,MonthlyCharges,TotalCharges,gender_Female,gender_Male,SeniorCitizen_0,SeniorCitizen_1,Partner_No,Partner_Yes,Dependents_No,Dependents_Yes,PhoneService_No,PhoneService_Yes,MultipleLines_No,MultipleLines_No phone service,MultipleLines_Yes,InternetService_DSL,InternetService_Fiber optic,InternetService_No,OnlineSecurity_No,OnlineSecurity_No internet service,OnlineSecurity_Yes,OnlineBackup_No,OnlineBackup_No internet service,OnlineBackup_Yes,DeviceProtection_No,DeviceProtection_No internet service,DeviceProtection_Yes,TechSupport_No,TechSupport_No internet service,TechSupport_Yes,StreamingTV_No,StreamingTV_No internet service,StreamingTV_Yes,StreamingMovies_No,StreamingMovies_No internet service,StreamingMovies_Yes,Contract_Month-to-month,Contract_One year,Contract_Two year,PaperlessBilling_No,PaperlessBilling_Yes,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,Churn
0,1,29.85,29.85,1,0,1,0,0,1,1,0,1,0,0,1,0,1,0,0,1,0,0,0,0,1,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0
1,34,56.95,1889.50,0,1,1,0,1,0,1,0,0,1,1,0,0,1,0,0,0,0,1,1,0,0,0,0,1,1,0,0,1,0,0,1,0,0,0,1,0,1,0,0,0,0,1,0
2,2,53.85,108.15,0,1,1,0,1,0,1,0,0,1,1,0,0,1,0,0,0,0,1,0,0,1,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,1,1
3,45,42.30,1840.75,0,1,1,0,1,0,1,0,1,0,0,1,0,1,0,0,0,0,1,1,0,0,0,0,1,0,0,1,1,0,0,1,0,0,0,1,0,1,0,1,0,0,0,0
4,2,70.70,151.65,1,0,1,0,1,0,1,0,0,1,1,0,0,0,1,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7027,24,84.80,1990.50,0,1,1,0,0,1,0,1,0,1,0,0,1,1,0,0,0,0,1,1,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,1,0,0,1,0,0,0,1,0
7028,72,103.20,7362.90,1,0,1,0,0,1,0,1,0,1,0,0,1,0,1,0,1,0,0,0,0,1,0,0,1,1,0,0,0,0,1,0,0,1,0,1,0,0,1,0,1,0,0,0
7029,11,29.60,346.45,1,0,1,0,0,1,0,1,1,0,0,1,0,1,0,0,0,0,1,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0
7030,4,74.40,306.60,0,1,0,1,0,1,1,0,0,1,0,0,1,0,1,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,1,1


In [ ]:
#df.to_csv('/content/gdrive/My Drive/TubesAI_folder/IBM_pp.csv',index=False)